In [92]:
import mysql.connector
from mysql.connector import errorcode
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
import pickle
import imdb_funcs
import json
import re

In [93]:
with open('cleaned_movie_data_for_50_actors.json', 'r') as read_file:
    data3 = json.load(read_file)

In [94]:
import db_funcs

In [17]:
help(db_funcs)

Help on module db_funcs:

NAME
    db_funcs

FUNCTIONS
    get_db_conn(config_params)
        get_db_conn(config_params):
        Open a MySQL DB connection using
        config parameters
        Params:
            config module: config_params
        Returns:
            MySQL DB connection or None if exception
    
    insert_actor_values(cursor, actor_id, first_name, last_name, pop_rank, social_rank, salary)
        insert_actor_values(cursor, actor_id, first_name, last_name, pop_rank,
                            social_rank, salary):
        Insert values into the actor table
        Params:
        Returns:
    
    insert_movie_values(cursor, title_id, title, director, studio, meta_score, budget, bo_gu, bo_ww, bo_ow)
        insert_movie_values(movie_id, title, director, studio, meta_score, budget,
                            bo_gu, bo_ww, bo_ow, config_params):
        
        Insert values into movie table
        Params:
        Returns:

FILE
    /Users/flatironschool/Docu

In [18]:
data3

[{'movie_id': 'tt5697510',
  'title': 'Jekyll',
  'director': 'Ruben Fleischer',
  'cast': ['Chris Evans',
   'Prime VideoUnlimited Streamingof Movies & TV',
   'Amazon FranceBuy Movies onDVD & Blu-ray',
   None,
   None,
   None],
  'genres': None,
  'metascore': 0,
  'budget': 0,
  'box office gross usa': 0,
  'box office cumulative': 0,
  'box office opening week': 0,
  'studio': ' A Very Good Production Inc.'},
 {'movie_id': 'tt8946378',
  'title': 'Knives Out (2019)',
  'director': 'Rian Johnson',
  'cast': ['Chris Evans',
   'Ana de Armas',
   'Daniel Craig',
   'Jamie Lee Curtis',
   'Michael Shannon',
   'Toni Collette'],
  'genres': ['Comedy', 'Crime', 'Drama', 'Mystery', 'Thriller'],
  'metascore': 0,
  'budget': 0,
  'box office gross usa': 0,
  'box office cumulative': 0,
  'box office opening week': 0,
  'studio': ' FilmNation Entertainment'},
 {'movie_id': 'tt4995776',
  'title': 'Operation Brothers (2019)',
  'director': 'Gideon Raff',
  'cast': ['Chris Evans',
   'Haley

In [19]:
# insert_movie_values(cursor, title_id, title, director, studio, meta_score,
#                        budget, bo_gu, bo_ww, bo_ow):

In [32]:
#sample_movies=data3[:10]

In [33]:
#sample_movies

[{'movie_id': 'tt5697510',
  'title': 'Jekyll',
  'director': 'Ruben Fleischer',
  'cast': ['Chris Evans',
   'Prime VideoUnlimited Streamingof Movies & TV',
   'Amazon FranceBuy Movies onDVD & Blu-ray',
   None,
   None,
   None],
  'genres': None,
  'metascore': 0,
  'budget': 0,
  'box office gross usa': 0,
  'box office cumulative': 0,
  'box office opening week': 0,
  'studio': ' A Very Good Production Inc.'},
 {'movie_id': 'tt8946378',
  'title': 'Knives Out (2019)',
  'director': 'Rian Johnson',
  'cast': ['Chris Evans',
   'Ana de Armas',
   'Daniel Craig',
   'Jamie Lee Curtis',
   'Michael Shannon',
   'Toni Collette'],
  'genres': ['Comedy', 'Crime', 'Drama', 'Mystery', 'Thriller'],
  'metascore': 0,
  'budget': 0,
  'box office gross usa': 0,
  'box office cumulative': 0,
  'box office opening week': 0,
  'studio': ' FilmNation Entertainment'},
 {'movie_id': 'tt4995776',
  'title': 'Operation Brothers (2019)',
  'director': 'Gideon Raff',
  'cast': ['Chris Evans',
   'Haley

In [95]:
import config

In [96]:
config.host

'flatiron.cscixofgkh2c.us-east-2.rds.amazonaws.com'

In [105]:
# get connection:
cnx = mysql.connector .connect(
    host = config.host,
    user = 'omar',
    passwd = config.password,
    database = 'movie_project'
)

In [39]:
# cnx.close()

In [106]:
print(cnx) # it will print a connection object if everything is find

In [42]:
# # in a loop:
# # for movie in movies and call func with values::
#     # get values
# def dynamic_entry_movies(movies):
#     cursor = cnx.cursor()
#     for movie in movies:
#         movie_id = movie['movie_id']
#         title = movie['title']
#         director = movie['director']
#         studio = movie['studio']
#         meta_score = movie['metascore']
#         budget = movie['budget']
#         bo_gu = movie['box office gross usa']
#         bo_ww = movie['box office cumulative']
#         bo_ow = movie['box office opening week']
    
#         db_funcs.insert_movie_values(cursor, movie_id, title, director, studio, meta_score, budget, bo_gu, bo_ww, bo_ow)


In [45]:
# dynamic_entry_movies(data3)

In [46]:
# cnx.commit()

In [47]:
# cnx.close()

In [ ]:
# junction movies+genres

In [98]:
genres_ids = ['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentry', 'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Thriller', 'War', 'Western']

In [99]:

def create_junc_mvxgn(genre_id, movie_list,):
    dummy_list = []
    for movie in movie_list:
        if movie['genres']:
            for genre in movie['genres']:
                if genre in genres_ids:
                    dummy_list.append({movie['movie_id']: genre})
    return dummy_list
                

In [107]:
list1 = create_junc_mvxgn(genres_ids, data3)

In [112]:
# in a loop:
# for movie in movies and call func with values::
    # get values
def dynamic_entry_genres(genres):
    cursor = cnx.cursor()
    for genre in genres:
        for k,v in genre.items():
            title_id = k
            genre = v
            
            query = """INSERT INTO moviesxgenres (title_id, genre) VALUES(%s, %s)"""
            values = (title_id, genre)
            cursor.execute(query, values)

In [113]:
dynamic_entry_genres(list1)

tt8946378
Comedy
tt8946378
Crime
tt8946378
Drama
tt8946378
Mystery
tt8946378
Thriller
tt4995776
Drama
tt4995776
History
tt4995776
Thriller
tt4154796
Action
tt4154796
Adventure
tt4154796
Sci-Fi
tt4154664
Action
tt4154664
Adventure
tt4154664
Sci-Fi
tt4154756
Action
tt4154756
Adventure
tt4154756
Sci-Fi
tt2250912
Action
tt2250912
Adventure
tt2250912
Sci-Fi
tt4481414
Drama
tt3498820
Action
tt3498820
Adventure
tt3498820
Sci-Fi
tt0478970
Action
tt0478970
Adventure
tt0478970
Comedy
tt0478970
Sci-Fi
tt2395427
Action
tt2395427
Adventure
tt2395427
Sci-Fi
tt2139555
Comedy
tt2139555
Romance
tt0443465
Comedy
tt0443465
Drama
tt0443465
Romance
tt1843866
Action
tt1843866
Adventure
tt1843866
Sci-Fi
tt1981115
Action
tt1981115
Adventure
tt1981115
Fantasy
tt1706620
Action
tt1706620
Drama
tt1706620
Sci-Fi
tt1491044
Biography
tt1491044
Crime
tt1491044
Drama
tt1491044
Thriller
tt0848228
Action
tt0848228
Adventure
tt0848228
Sci-Fi
tt0770703
Comedy
tt0770703
Romance
tt1740721
Action
tt1740721
Adventure
tt174072

tt2850386
Family
tt2850386
Fantasy
tt6264654
Action
tt6264654
Adventure
tt6264654
Comedy
tt6264654
Sci-Fi
tt8385148
Action
tt8385148
Comedy
tt8385148
Thriller
tt8106534
Action
tt8106534
Thriller
tt5884052
Action
tt5884052
Adventure
tt5884052
Comedy
tt5884052
Family
tt5884052
Mystery
tt5884052
Sci-Fi
tt8944244
Short
tt8944244
Comedy
tt5463162
Action
tt5463162
Adventure
tt5463162
Comedy
tt5463162
Sci-Fi
tt1959563
Action
tt1959563
Comedy
tt1959563
Thriller
tt5442430
Horror
tt5442430
Sci-Fi
tt5442430
Thriller
tt6612630
Short
tt6612630
Comedy
tt6612630
Crime
tt6273870
Action
tt3014866
Action
tt3014866
Crime
tt3014866
Drama
tt3014866
Mystery
tt3014866
Sci-Fi
tt3014866
Thriller
tt1431045
Action
tt1431045
Adventure
tt1431045
Comedy
tt1431045
Sci-Fi
tt2140379
Action
tt2140379
Mystery
tt2140379
Sci-Fi
tt2140379
Thriller
tt2404425
Biography
tt2404425
Drama
tt2404425
History
tt2349144
Comedy
tt2349144
Drama
tt2326612
Crime
tt2326612
Drama
tt2326612
Mystery
tt2326612
Thriller
tt2557490
Comedy
tt255

Drama
tt0210719
Romance
tt0124315
Drama
tt0124315
Romance
tt0137338
Comedy
tt0137338
Drama
tt0137338
Romance
tt0171854
Comedy
tt0171854
Drama
tt0171854
Romance
tt0120772
Comedy
tt0120772
Drama
tt0120772
Romance
tt0117276
Comedy
tt0117276
Romance
tt0119557
Drama
tt8608518
Drama
tt0117509
Drama
tt0117509
Romance
tt0117660
Comedy
tt0117660
Music
tt0113253
Horror
tt0113253
Thriller
tt0112697
Comedy
tt0112697
Romance
tt0107592
Drama
tt0107592
Crime
tt0105998
Adventure
tt0105998
Drama
tt0105998
Sci-Fi
tt3404006
Short
tt3404006
Drama
tt0282627
Drama
tt4244994
Drama
tt7068946
Action
tt7068946
Crime
tt7068946
Drama
tt5981922
Crime
tt5981922
Drama
tt5981922
Mystery
tt5981922
Thriller
tt4781650
Drama
tt4781650
War
tt7456312
Crime
tt7456312
Drama
tt7456312
Mystery
tt7456312
Thriller
tt8544498
Drama
tt8544498
Sport
tt6521876
Comedy
tt1488606
Action
tt1488606
Adventure
tt1488606
Crime
tt1488606
Drama
tt1488606
Thriller
tt0974015
Action
tt0974015
Adventure
tt0974015
Fantasy
tt0974015
Sci-Fi
tt2361317

tt5460530
Action
tt5460530
Crime
tt5460530
Drama
tt5460530
Thriller
tt5580536
Action
tt5580536
Crime
tt5580536
Thriller
tt4382824
Adventure
tt4382824
Biography
tt4382824
Comedy
tt2032572
Action
tt2032572
Drama
tt2032572
History
tt2032572
Thriller
tt2032572
War
tt3774114
Biography
tt3774114
Drama
tt3774114
Thriller
tt4054654
Action
tt4054654
Thriller
tt3733774
Crime
tt3733774
Drama
tt3733774
Thriller
tt3733778
Drama
tt3733778
Horror
tt3733778
Mystery
tt3733778
Thriller
tt3687398
Drama
tt1274586
Drama
tt1274586
Thriller
tt2467046
Action
tt2467046
Drama
tt2467046
Fantasy
tt2467046
Mystery
tt2467046
Thriller
tt1552224
Action
tt1552224
Adventure
tt1552224
Drama
tt1552224
Thriller
tt2401807
Action
tt2401807
Crime
tt2401807
Thriller
tt2382396
Crime
tt2382396
Drama
tt2005374
Crime
tt2005374
Drama
tt2005374
Mystery
tt2005374
Thriller
tt0481499
Animation
tt0481499
Action
tt0481499
Adventure
tt0481499
Comedy
tt0481499
Family
tt0481499
Fantasy
tt1656186
Action
tt1656186
Crime
tt1656186
Thriller
tt

tt0099685
Biography
tt0099685
Crime
tt0099685
Drama
tt0099528
Drama
tt0099528
Horror
tt0099528
Mystery
tt0100168
Drama
tt0100168
Music
tt0100168
Romance
tt0099128
Comedy
tt0099128
Romance
tt0099395
Horror
tt0100602
Comedy
tt0100602
Crime
tt0100602
Thriller
tt0098273
Crime
tt0098273
Drama
tt0098273
Mystery
tt0098273
Romance
tt0098273
Thriller
tt0097216
Comedy
tt0097216
Drama
tt0097164
Crime
tt0097164
Drama
tt0094898
Comedy
tt0094898
Romance
tt0096054
Comedy
tt0096054
Drama
tt0096054
Musical
tt0093474
Comedy
tt0094213
Drama
tt0082970
Drama
tt0080707
Action
tt0080707
Crime
tt0080707
Drama
tt0080707
Thriller
tt0217105
Drama
tt0074413
Drama
tt0202001
Drama
tt7428530
Biography
tt7428530
Drama
tt7428530
History
tt5537002
Crime
tt5537002
Drama
tt5537002
History
tt5537002
Mystery
tt5537002
Thriller
tt6487156
Drama
tt7131622
Comedy
tt7131622
Drama
tt1663202
Action
tt1663202
Adventure
tt1663202
Biography
tt1663202
Drama
tt1663202
Western
tt4016250
Short
tt4016250
Comedy
tt0993846
Biography
tt0993

Comedy
tt1234654
Drama
tt1234654
Romance
tt2095802
Short
tt1526300
Animation
tt1526300
Short
tt1526300
Adventure
tt1526300
Comedy
tt1526300
Family
tt1526300
Fantasy
tt0913413
Comedy
tt0913413
Drama
tt0913413
Music
tt1078912
Adventure
tt1078912
Comedy
tt1078912
Family
tt1078912
Fantasy
tt1425242
Action
tt1425242
Adventure
tt1425242
Comedy
tt1425242
Family
tt1425242
Fantasy
tt0479952
Animation
tt0479952
Adventure
tt0479952
Comedy
tt0479952
Family
tt0942385
Action
tt0942385
Adventure
tt0942385
Comedy
tt0942385
War
tt1077081
Family
tt1077081
Music
tt0408839
Comedy
tt0408839
Romance
tt0477347
Adventure
tt0477347
Comedy
tt0477347
Family
tt0477347
Fantasy
tt0365830
Adventure
tt0365830
Comedy
tt0365830
Music
tt0365830
Musical
tt0462519
Comedy
tt0470995
Comedy
tt1449273
Comedy
tt0351283
Animation
tt0351283
Adventure
tt0351283
Comedy
tt0351283
Family
tt0337887
Action
tt0337887
Comedy
tt0337887
Musical
tt0290002
Comedy
tt0290002
Romance
tt0357413
Comedy
tt0364725
Comedy
tt0364725
Sport
tt0326856


tt1524137
Drama
tt1524137
Thriller
tt0964517
Biography
tt0964517
Drama
tt0964517
Sport
tt1386588
Action
tt1386588
Comedy
tt1386588
Crime
tt1279935
Comedy
tt1279935
Crime
tt1279935
Romance
tt1279935
Thriller
tt0380510
Drama
tt0380510
Fantasy
tt0380510
Thriller
tt0467197
Action
tt0467197
Crime
tt0467197
Drama
tt0467197
Mystery
tt0467197
Thriller
tt0949731
Adventure
tt0949731
Drama
tt0949731
Mystery
tt0949731
Sci-Fi
tt0949731
Thriller
tt0498399
Crime
tt0498399
Drama
tt0498399
Thriller
tt0822854
Action
tt0822854
Drama
tt0822854
Thriller
tt0407887
Crime
tt0407887
Drama
tt0407887
Thriller
tt0445990
Biography
tt0445990
Drama
tt0445990
Sport
tt0430105
Action
tt0430105
Crime
tt0430105
Drama
tt0430105
Mystery
tt0430105
Thriller
tt0356721
Comedy
tt0317740
Action
tt0317740
Crime
tt0317740
Thriller
tt0270707
Mystery
tt0270707
Thriller
tt0202470
Drama
tt0202470
Music
tt0133152
Action
tt0133152
Adventure
tt0133152
Sci-Fi
tt0133152
Thriller
tt0177971
Action
tt0177971
Adventure
tt0177971
Drama
tt017797

In [114]:
cnx.commit()

In [115]:
cnx.close()